See more at: https://pandas-gbq.readthedocs.io/en/latest/intro.html#logging

In [1]:
# Required files
import os
import sys
import pandas as pd
import pandas_gbq
import re

# Connect colab with google bigquery stuff
from google.colab import auth
from google.cloud import bigquery
from google.colab import files
from google.colab import drive

**Authenticate:** The line of code below ensures you are an authenticated user accessing the MIMIC database. You will need to rerun this each time you open the notebook.

In [2]:
auth.authenticate_user()

**Query Function:** This is a method that executes a desired SQL query on the database. If you want to run a query, you can use the function name below, which we named run_query()

In [3]:
!jupyter-kernelspec list
# This line activate R magic
%load_ext rpy2.ipython

Available kernels:
  ir         /usr/local/share/jupyter/kernels/ir
  python3    /usr/local/share/jupyter/kernels/python3


In [4]:
# TODO:
# Specify your project_id on Google Cloud
project_id='test-project-mimic'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [6]:
# Read csv-sheet with ICD9_codes
%%R
#install.packages("readr")
library(tidyverse)
library(readr)
ICD9_Codes <- read_csv("/content/ICD9_Codes_for_Cohort_Alcohol - Sheet1.csv", col_types = "ccc") #%>% select("ICD9_Code") %>% pull()
ICD9_Codes

# A tibble: 35 × 5
   Cohort_Number Cohort_Name ICD9_Code ICD9_Text                         Added…¹
   <chr>         <chr>       <chr>     <chr>                             <chr>  
 1 1             Alcohol     30500     Alcohol abuse, unspecified        Malte  
 2 1             Alcohol     30390     Other and unspecified alcohol de… Malte  
 3 1             Alcohol     29181     Alcohol withdrawal                Malte  
 4 1             Alcohol     5711      Acute alcoholic hepatitis         Malte  
 5 1             Alcohol     30301     Acute alcoholic intoxication in … Malte  
 6 1             Alcohol     30391     Other and unspecified alcohol de… Malte  
 7 1             Alcohol     30501     Alcohol abuse, continuous         Malte  
 8 1             Alcohol     2910      Alcohol withdrawal delirium       Malte  
 9 1             Alcohol     53531     Alcoholic gastritis, with hemorr… Malte  
10 1             Alcohol     30300     Acute alcoholic intoxication in … Malte  
# … with 

In [7]:
import numpy as np
import rpy2.robjects as robjects

#pd.DataFrame(robjects.globalenv['ICD9_Codes'])

ICD9_codes_by_cohort =pd.read_csv("/content/ICD9_Codes_for_Cohort_Alcohol - Sheet1.csv")
#ICD9_array = np.array(robjects.globalenv['ICD9_Codes']).tolist()


In [8]:
unique_cohorts = list(set(ICD9_codes_by_cohort["Cohort_Name"]))
output_dict = {}

ICD9_dict = {x : ICD9_codes_by_cohort.loc[ICD9_codes_by_cohort["Cohort_Name"] == x,"ICD9_Code"] for x in unique_cohorts}

In [18]:
ICD9_array = ICD9_codes_by_cohort["ICD9_Code"].values

In [32]:
output_dict = {}
for cohort in unique_cohorts:
  current_cohort_ICD9_array = ICD9_codes_by_cohort.loc[ICD9_codes_by_cohort["Cohort_Name"]== cohort, "ICD9_Code"].values

  query = '''
  SELECT DISTINCT dx.HADM_ID 
  FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses` AS icd
  INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` AS dx
    ON dx.ICD9_CODE = icd.ICD9_CODE
  WHERE dx.ICD9_CODE IN ({})
  '''

  placeholders = ', '.join(['"{}"'] * len(current_cohort_ICD9_array))
  query = query.format(placeholders)
  query = query.format(*current_cohort_ICD9_array)

  Cohort_hadm_ids = (pandas_gbq.read_gbq(query, project_id=project_id, dialect = 'standard'))

  Cohort_hadm_ids.to_csv(cohort + ".csv", index = True)
  #output_dict[cohort] = Cohort_hadm_ids



Downloading: 100%|██████████| 255/255 [00:00<00:00, 1571.08rows/s]


In [30]:
output_dict

{'Smoking': array([[156068],
        [137131],
        [117940],
        ...,
        [120262],
        [150891],
        [138742]]), 'Hypochondriacs': array([[114447],
        [122287],
        [156892],
        [178492],
        [159651]]), 'Alcohol': array([[171514],
        [193644],
        [156068],
        ...,
        [166101],
        [126633],
        [149738]]), 'Physical Disability': array([[128268],
        [124945],
        [195795],
        [115598],
        [156100],
        [162452],
        [121545],
        [179743],
        [162198],
        [104896],
        [171423],
        [130711],
        [160831],
        [169901],
        [169761],
        [192478],
        [184682],
        [162451],
        [134238],
        [191217],
        [165525],
        [182008],
        [156698],
        [186077],
        [196634],
        [134921],
        [183327],
        [182659],
        [160147],
        [196594],
        [184649],
        [183768],
        [140633],
        

In [19]:
# Your regular sql query

# For example:
query = '''
SELECT DISTINCT dx.HADM_ID 
FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses` AS icd
INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` AS dx
  ON dx.ICD9_CODE = icd.ICD9_CODE
WHERE dx.ICD9_CODE IN ({})
'''

placeholders = ', '.join(['"{}"'] * len(ICD9_array))
query = query.format(placeholders)
query = query.format(*ICD9_array)

In [20]:
# Then directly query mimic data -- takes about a minute to run
Cohort_1_Alcohol = pandas_gbq.read_gbq(query, project_id=project_id, dialect = 'standard')

Downloading: 100%|██████████| 9474/9474 [00:00<00:00, 26997.52rows/s]


In [23]:
# Lets take a look at the dataframe
Cohort_1_Alcohol.values

array([[114447],
       [122287],
       [156892],
       ...,
       [189003],
       [120262],
       [138742]])

# Exported derived dataset back to BiqQuery for later analysis

In [15]:
# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
Cohort_1_Alcohol.to_csv("Alcohol.csv", index = False)

#pandas_gbq.to_gbq(CXRreports, table_id, project_id=project_id)